In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import os
import cv2
from glob import glob
import random
import torch
from torch.utils.data import Dataset, DataLoader
from tqdm.notebook import tqdm
from torch import nn
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, recall_score, precision_score, f1_score

In [2]:
random.seed(42)
np.random.seed(42)
torch.manual_seed(0)

In [3]:
df = pd.read_csv("../datasets/waterbird/metadata.csv")
df.head(5)

,img_id,img_filename,y,split,place,place_filename
0,1,001.Black_footed_Albatross/Black_Footed_Albatr...,1,2,1,/o/ocean/00002178.jpg
1,2,001.Black_footed_Albatross/Black_Footed_Albatr...,1,0,1,/l/lake/natural/00000065.jpg
2,3,001.Black_footed_Albatross/Black_Footed_Albatr...,1,2,0,/b/bamboo_forest/00000131.jpg
3,4,001.Black_footed_Albatross/Black_Footed_Albatr...,1,0,1,/o/ocean/00001268.jpg
4,5,001.Black_footed_Albatross/Black_Footed_Albatr...,1,0,1,/o/ocean/00003147.jpg


In [4]:
LANDBIRD = 0
WATERBIRD = 1
LAND = 0
WATER = 1

In [12]:
performance_report = np.load("D:\Download\Waterbird\PaliGemma\landbird_or_waterbird\Trained\lr_00005\landbird_or_waterbird_train.npy", allow_pickle=True)

In [13]:
true_labels = [x['img_label'] for x in performance_report]
predicted_labels = [x['predicted'] for x in performance_report]
place = [x['place'] for x in performance_report]

In [14]:
landbird_on_land = []
landbird_on_water = []
waterbird_on_land = []
waterbird_on_water = []

predicted_landbird_on_land = []
predicted_landbird_on_water = []
predicted_waterbird_on_land = []
predicted_waterbird_on_water = []

predicted_landbird_on_land_correct = []
predicted_landbird_on_land_incorrect = []

predicted_landbird_on_water_correct = []
predicted_landbird_on_water_incorrect = []

predicted_waterbird_on_land_correct = []
predicted_waterbird_on_land_incorrect = []

predicted_waterbird_on_water_correct = []
predicted_waterbird_on_water_incorrect = []

for report in performance_report:
    if report['img_label'] == LANDBIRD and report['place'] == LAND:
        landbird_on_land.append(report['img_label'])
        predicted_landbird_on_land.append(report['predicted'])
        if report['img_label'] == report['predicted']:
            predicted_landbird_on_land_correct.append(report)
        else:
            predicted_landbird_on_land_incorrect.append(report)

    elif report['img_label'] == LANDBIRD and report['place'] == WATER:
        landbird_on_water.append(report['img_label'])
        predicted_landbird_on_water.append(report['predicted'])
        if report['img_label'] == report['predicted']:
            predicted_landbird_on_water_correct.append(report)
        else:
            predicted_landbird_on_water_incorrect.append(report)

    elif report['img_label'] == WATERBIRD and report['place'] == LAND:
        waterbird_on_land.append(report['img_label'])
        predicted_waterbird_on_land.append(report['predicted'])
        if report['img_label'] == report['predicted']:
            predicted_waterbird_on_land_correct.append(report)
        else:
            predicted_waterbird_on_land_incorrect.append(report)

    elif report['img_label'] == WATERBIRD and report['place'] == WATER:
        waterbird_on_water.append(report['img_label'])
        predicted_waterbird_on_water.append(report['predicted'])
        if report['img_label'] == report['predicted']:
            predicted_waterbird_on_water_correct.append(report)
        else:
            predicted_waterbird_on_water_incorrect.append(report)

In [15]:
test_acc = accuracy_score(true_labels, predicted_labels)
landbird_on_land_acc = accuracy_score(landbird_on_land, predicted_landbird_on_land)
landbird_on_water_acc = accuracy_score(landbird_on_water, predicted_landbird_on_water)
waterbird_on_water_acc = accuracy_score(waterbird_on_water, predicted_waterbird_on_water)
waterbird_on_land_acc = accuracy_score(waterbird_on_land, predicted_waterbird_on_land)

test_f1 = f1_score(true_labels, predicted_labels)
landbird_on_land_f1 = f1_score(landbird_on_land, predicted_landbird_on_land)
landbird_on_water_f1 = f1_score(landbird_on_water, predicted_landbird_on_water)
waterbird_on_water_f1 = f1_score(waterbird_on_water, predicted_waterbird_on_water)
waterbird_on_land_f1 = f1_score(waterbird_on_land, predicted_waterbird_on_land)

print(f"All data\nAccuracy: {test_acc:.2f} | F1 Score: {test_f1:.2f}")
print(f"Landbird on land\nAccuracy: {landbird_on_land_acc:.2f} | F1 Score: {landbird_on_land_f1:.2f}")
print(f"Landbird on water\nAccuracy: {landbird_on_water_acc:.2f} | F1 Score: {landbird_on_water_f1:.2f}")
print(f"Waterbird on water\nAccuracy: {waterbird_on_water_acc:.2f} | F1 Score: {waterbird_on_water_f1:.2f}")
print(f"Waterbird on land\nAccuracy: {waterbird_on_land_acc:.2f} | F1 Score: {waterbird_on_land_f1:.2f}")

All data
Accuracy: 0.80 | F1 Score: 0.45
Landbird on land
Accuracy: 0.99 | F1 Score: 0.00
Landbird on water
Accuracy: 0.98 | F1 Score: 0.00
Waterbird on water
Accuracy: 0.30 | F1 Score: 0.46
Waterbird on land
Accuracy: 0.31 | F1 Score: 0.47


In [10]:
# print("Test set")
# print(classification_report(
#     true_labels, predicted_labels, output_dict=False))
# print("Landbird on land")
# print(classification_report(
#     landbird_on_land, predicted_landbird_on_land, zero_division=True, output_dict=False))
# print("Landbird on water")
# print(classification_report(
#     landbird_on_water, predicted_landbird_on_water, zero_division=True, output_dict=False))
# print("Waterbird on water")
# print(classification_report(
#     waterbird_on_water, predicted_waterbird_on_water, zero_division=True, output_dict=False))
# print("Waterbird on land")
# print(classification_report(
#     waterbird_on_land, predicted_waterbird_on_land, zero_division=True, output_dict=False))

Test set
              precision    recall  f1-score   support

           0       0.83      0.97      0.90      4510
           1       0.75      0.30      0.43      1284

    accuracy                           0.82      5794
   macro avg       0.79      0.64      0.66      5794
weighted avg       0.81      0.82      0.79      5794

Landbird on land
              precision    recall  f1-score   support

           0       1.00      0.97      0.99      2255
           1       0.00      1.00      0.00         0

    accuracy                           0.97      2255
   macro avg       0.50      0.99      0.49      2255
weighted avg       1.00      0.97      0.99      2255

Landbird on water
              precision    recall  f1-score   support

           0       1.00      0.97      0.98      2255
           1       0.00      1.00      0.00         0

    accuracy                           0.97      2255
   macro avg       0.50      0.98      0.49      2255
weighted avg       1.00      0